## C03 Making Fast Function Calls
### Using Globals
> Don't use globals in performance critical parts of code

#### The trouble with globals
Compiler has been unable to infer the type of result when working with the global variable, marking it as `Any`

In [1]:
using BenchmarkTools

In [2]:
p = 2
function pow_array(x::Vector{Float64})
    s = 0.0
    for y in x
        s += y^p
    end
    return s
end

pow_array (generic function with 1 method)

In [3]:
t = rand(100000);
@btime pow_array(t);

  3.821 ms (300000 allocations: 4.58 MiB)


In [4]:
@code_warntype pow_array(t)

MethodInstance for pow_array(::Vector{Float64})
  from pow_array(x::Vector{Float64}) @ Main ~/code/Julia-HPC/03-Functions.ipynb:2
Arguments
  #self#

::Core.Const(pow_array)
  x::Vector{Float64}
Locals
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  s::Any
  y::Float64
Body::Any
1 ─

(s = 0.0)


│   %2  = x::Vector{Float64}
│         (@_3 = Base.iterate(%

2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Float64, Int64}
│         (y = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = s::Any
│   %11 = (y ^ Main.p)::Any
│         (s = %10 + %11)
│         (@_3 = Base.iterate(%2, %9))
│   %14 = (@_3 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return s



#### Fixing performance issues with globals
There are two ways to correct the issues of globals
1. `const p`: which will constrain the type of variable p.
2. `pow(array::Vector{Float64}, p)`: make the `p` being the argument of function, during which the type of `p` will be inference.

In [5]:
const p2 = 2
function pow_array2(x::Vector{Float64})
    s = 0.0
    for y in x
        s += y^p2
    end
    return s
end

pow_array2 (generic function with 1 method)

In [6]:
@btime pow_array2(t);

  443.834 μs (0 allocations: 0 bytes)


In [7]:
@code_warntype pow_array2(t)

MethodInstance for pow_array2(::Vector{Float64})
  from pow_array2(x::Vector{Float64}) @ Main ~/code/Julia-HPC/03-Functions.ipynb:2
Arguments
  #self#::Core.Const(pow_array2)
  x::Vector{Float64}
Locals
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  s::Float64
  y::Float64
Body::Float64
1 ─       (s = 0.0)
│   %2  = x::Vector{Float64}
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Float64, Int64}
│         (y = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = s::Float64
│   %11 = (y ^ Main.p2)::Float64
│         (s = %10 + %11)
│         (@_3 = Base.iterate(%2, %9))
│   %14 = (@_3 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return s



In [8]:
function pow_array3(x::Vector{Float64})
    return pow_array_inner(x, p)
end

function pow_array_inner(x, pow)
    s = 0.0
    for y in x
        s += y^pow
    end
    return s
end

pow_array_inner (generic function with 1 method)

In [9]:
@btime pow_array3(t);

  313.258 μs (1 allocation: 16 bytes)


### Inlining
#### Default Inlining


In [10]:
function f(x)
    a = x * 5
    b = a + 3
end
g(x) = f(2 * x)

g (generic function with 1 method)

In [11]:
@code_typed g(3)

CodeInfo(
1 ─ %1 = Base.mul_int(2, x)::Int64
│   %2 = Base.mul_int(%1, 5)::Int64
│   %3 = Base.add_int(%2, 3)::Int64
└──      return %3
) => Int64

In [12]:
@code_llvm g(3)

;  @ /home/zpp/code/Julia-HPC/03-Functions.ipynb:5 within `g`
define 

i64 @julia_g_1771(i64 signext %0) #0 {
top:
; ┌ @ /home/zpp/code/Julia-HPC/03-Functions.ipynb:2 within `f`
; │┌ @ int.jl:88 within `*`
    %1 

= mul i64 %0, 10
; │└
; │ @ /home/zpp/code/Julia-HPC/03-Functions.ipynb:3 within `f`
; │┌ @ int.jl:87 within `+`
    %2 = add i64 %1, 3
; └└
  ret i64 %2
}


#### Controlling inlining
- `@inline`: make function be inlining
- `@noinline`: make function not be inlining

In [13]:
@noinline function f(x)
    a = x * 5
    b = a + 3
    c = a - 4
    if c < 0
        throw(DomainError())
    elseif c < 2
        d = c^3
    else
        d = c^2
    end
end


f (generic function with 1 method)

In [14]:
@code_typed g(3)

CodeInfo(
1 ─ %1 = Base.mul_int(2, x)::Int64
│   %2 = invoke Main.f(%1::Int64)::Int64
└──      return %2
) => Int64

In [15]:
@inline function f_in(x)
    a = x * 5
    b = a + 3
    c = a - 4
    if c < 0
        throw(DomainError())
    elseif c < 2
        d = c^3
    else
        d = c^2
    end
end

g_in(x) = f_in(2 * x)

g_in (generic function with 1 method)

In [16]:
@code_typed g_in(3)

CodeInfo(
1 ─ %1  = Base.mul_int(2, x)::Int64
│   %2  = Base.mul_int(%1, 5)::Int64
│   %3  = Base.sub_int(%2, 4)::Int64
│   %4  = Base.slt_int(%3, 0)::Bool
└──       goto #3 if not %4
2 ─       Main.DomainError()::Union{}
└──       unreachable
3 ─ %8  = Base.slt_int(%3, 2)::Bool
└──       goto #5 if not %8
4 ─ %10 = Base.mul_int(%3, %3)::Int64
│   %11 = Base.mul_int(%10, %3)::Int64
└──       goto #6
5 ─ %13 = Base.mul_int(%3, %3)::Int64
└──       goto #6
6 ┄ %15 = φ (#4 => %11, #5 => %13)::Int64
└──       return %15
) => Int64

In [17]:
@btime g(3);
@btime g_in(3);

  1.263 ns (0 allocations: 0 bytes)


  1.265 ns (0 allocations: 0 bytes)


### Constant propagation


In [18]:
sqr(x) = x * x;
sqr2() = sqr(2)

sqr2 (generic function with 1 method)

In [19]:
@code_typed sqr(2)

CodeInfo(
1 ─ %1 = Base.mul_int(x, x)::Int64
└──      return %1
) => Int64

In [20]:
@code_typed sqr2()

CodeInfo(
1 ─     return 4
) => Int64

### Using marcos for performance
#### The Julia compilation process
<img src="https://new-pic-zpp.oss-cn-guangzhou.aliyuncs.com/pic/202401161046940.png" width=650/>

#### Using marcos
- Marcos are code that can be used to write other code. A marco is executed very early in the compilation process.

#### Evaluating a polynomial
$$
p(x) = \sum_{i=0}^{n} a_{i} x^{i} = a_{0} + a_{1}x + a_{2}x^{2} + \cdots + a_{n}x^{n}
$$

In [21]:
function poly_naive(x, a...)
    p = zero(x)
    for i in 1:length(a)
        p = p + a[i] * x^(i-1)
    end
    return p
end

poly_naive (generic function with 1 method)

In [22]:
f_naive(x) = poly_naive(x, 1,2,3,4,5,6,7,8,9)

f_naive (generic function with 1 method)

In [23]:
x = 3.5
@btime f_naive(x)

  45.220 ns (1 allocation: 16 bytes)


271125.95703125

In [24]:
@code_warntype f_naive(x)

MethodInstance for f_naive(::Float64)
  from f_naive(x) @ Main ~/code/Julia-HPC/03-Functions.ipynb:1
Arguments
  #self#::Core.Const(f_naive)
  x::Float64
Body::Float64
1 ─ %1 = Main.poly_naive(x, 1, 2, 3, 4, 5, 6, 7, 8, 9)::Float64
└──      return %1



In [25]:
peakflops() * 52e-9

4638.344324986463

#### Horner's method
$$
\begin{split}
b_{n} & = a_{n} \\
b_{n-1} & =  a_{n-1} + b_{n} x \\
b_{n-2} & =  a_{n-2} + b_{n-1} x \\
& \vdots \\
b_{0} & =  a_{0} + b_{1}x
\end{split}
$$


In [26]:
function poly_horner(x, a...)
    b = zero(x); 
    for i in length(a):-1:1
        b = a[i] + b * x
    end
    return b
end

poly_horner (generic function with 1 method)

In [27]:
f_hornor(x) =  poly_horner(x, 1,2,3,4,5,6,7,8,9)

f_hornor (generic function with 1 method)

In [28]:
@btime f_hornor($x)

  17.771 ns (0 allocations: 0 bytes)


271125.95703125

#### The Horner marco
- The coefficients of the polynomial are constants. They don't change and are known when writing the program. Could write as 
`muladd(x, muladd(x, muladd(x, muladd(x, ..., 4), 3), 2), 1)`

In [29]:
macro horner(x, p...)
    ex = esc(p[end])
    for i in length(p)-1:-1:1
        ex = :(muladd(t, $ex, $(esc(p[i]))))
    end
    Expr(:block, :(t = $(esc(x))), ex)
end

@horner (macro with 1 method)

In [30]:
f_horner_macro(x) = @horner(x, 1,2,3,4,5,6,7,8,9)

f_horner_macro (generic function with 1 method)

In [31]:
@btime f_horner_macro($x)

  2.538 ns (0 allocations: 0 bytes)


271125.95703125

### Generated functions
#### Using generated functions for performance
- `@generated`

In [32]:
function prod_dim(x::Array{T, N}) where {T,N}
    s = 1;
    for i in 1:N
        s = s * size(x,i)
    end
    return s
end

prod_dim (generic function with 1 method)

In [33]:
prod_dim(rand(10,5,5))

250

In [34]:
@generated function prod_dim_gen(x::Array{T,N}) where {T,N}
    ex = :(1);
    for i in 1:N
        ex = :(size(x, $i) * $ex)
    end
    return ex
end

prod_dim_gen (generic function with 1 method)

In [35]:
prod_dim_gen(rand(10,5,5))

250

In [36]:
function prod_dim_gen_impl(x::Array{T, N}) where {T,N}
    ex = :(1)
    for i in 1:N
       ex = :(size(x, $i) * $ex)
    end
    return ex
end

prod_dim_gen_impl (generic function with 1 method)

In [37]:
prod_dim_gen_impl(rand(10,5,5))

:(size(x, 3) * (size(x, 2) * (size(x, 1) * 1)))

In [38]:
@generated function polynonial_gen(x, args...)
    ex = :(args[end] * one(x));
    
    for i in length(args)-1:-1:1
        ex = :($ex * x + args[$i])
    end
    return ex
end

polynonial_gen (generic function with 1 method)

In [39]:
x = 3.5
xs = range(0, 10, length=100)
cc = vcat(1.0, [1/factorial(i) for i in 1:20]...)
fig = Figure()
ax = Axis(fig[1,1])
lines!(ax,xs,polynonial_gen.(xs, cc...))
lines!(ax, xs, exp, linestyle=:dash)
fig

UndefVarError: UndefVarError: `Figure` not defined

In [40]:
@btime polynonial_gen($x, 1, 1, 1/factorial(2), 1/factorial(3), 1/factorial(4), 1/factorial(5), 1/factorial(6), 1/factorial(7), 1/factorial(8));

  21.532 ns (0 allocations: 0 bytes)


In [41]:
@btime poly_horner($x, $cc...);

  351.500 ns (24 allocations: 384 bytes)


In [42]:
f_m_horner(x) = @horner(x, 1, 1, 1/factorial(2), 1/factorial(3), 1/factorial(4), 1/factorial(5), 1/factorial(6), 1/factorial(7), 1/factorial(8))

f_m_horner (generic function with 1 method)

In [43]:
@btime f_m_horner($x)

  8.801 ns (0 allocations: 0 bytes)


32.78848130967882

In [44]:
macro horner_myself(x, args...)
    es = (esc(args[end]));
    for i in length(args)-1:-1:1
        es = :($es * t + $(esc(args[i])))
    end
    return Expr(:block, :(t = $(esc(x))), es)
end

@horner_myself (macro with 1 method)

In [45]:
@generated function horner_gen2(x, args...)
    es = :(args[end]);
    for i in length(args)-1:-1:1
        es = :($es * t + args[$i])
    end
    return Expr(:block, :(t = x), es)
end

horner_gen2 (generic function with 1 method)

In [46]:
horner_gen2(x, 1, 1, 1/factorial(2), 1/factorial(3), 1/factorial(4), 1/factorial(5), 1/factorial(6), 1/factorial(7), 1/factorial(8))

32.78848130967882

In [47]:
args = [1:10...]
for i in args
    global es = :($(args[end]));
    for j in length(args)-1:-1:1
        es = :($es * t + $(args[j]))
    end
    println(es)

# :(x * 1)
end

((((((((10t + 9) * t + 8) * t + 7) * t + 6) * t + 5) * t + 4) * t + 3) * t + 2) * t + 1
((((((((10t + 9) * t + 8) * t + 7) * t + 6) * t + 5) * t + 4) * t + 3) * t + 2) * t + 1
((((((((10t + 9) * t + 8) * t + 7) * t + 6) * t + 5) * t + 4) * t + 3) * t + 2) * t + 1
((((((((10t + 9) * t + 8) * t + 7) * t + 6) * t + 5) * t + 4) * t + 3) * t + 2) * t + 1
((((((((10t + 9) * t + 8) * t + 7) * t + 6) * t + 5) * t + 4) * t + 3) * t + 2) * t + 1
((((((((10t + 9) * t + 8) * t + 7) * t + 6) * t + 5) * t + 4) * t + 3) * t + 2) * t + 1
((((((((10t + 9) * t + 8) * t + 7) * t + 6) * t + 5) * t + 4) * t + 3) * t + 2) * t + 1
((((((((10t + 9) * t + 8) * t + 7) * t + 6) * t + 5) * t + 4) * t + 3) * t + 2) * t + 1
((((((((10t + 9) * t + 8) * t + 7) * t + 6) * t + 5) * t + 4) * t + 3) * t + 2) * t + 1
((((((((10t + 9) * t + 8) * t + 7) * t + 6) * t + 5) * t + 4) * t + 3) * t + 2) * t + 1


In [48]:
f(x) = @horner_myself(x ,1,2,3,4)

f (generic function with 1 method)

In [49]:
eval(Expr(:block, :(t = 10), es))

10987654321

In [51]:
f(3.4)

199.696